<a href="https://colab.research.google.com/github/AbsoluteZer0527/De-clutter/blob/main/NN_Workshop_Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
from keras.datasets import mnist
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
#from skimage.color import rgb2gray
#from skimage.transform import resize
from ipywidgets import Output, Button
from IPython.display import Image
import random
import warnings
warnings.filterwarnings('ignore')

***NOTE***: No worries if the cell above shows a warning
Additionally you don't need to worry about the below cells

In [ ]:
#HELPER FUNCTIONS

def softmax(x):
    if len(x.shape) == 1:
        e_x = np.exp(x - np.max(x))
        return e_x / np.sum(e_x)
    else:
        e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return e_x / np.sum(e_x, axis=1, keepdims=True)



def relu(x):
    return np.maximum(0, x)


def relu_derivative(x):
    return np.where(x > 0, 1, 0)




def cross_entropy_loss(y_true, y_pred):
    n_samples = y_true.shape[0]
    logp = - np.log(y_pred[range(n_samples), y_true.argmax(axis=1)])
    loss = np.sum(logp) / n_samples
    return loss


def train_neural_network(df, hidden_size=700, output_size=10, lr=0.01, epochs=300):
    X = df.iloc[:, :-1].values
    y = pd.get_dummies(df.iloc[:, -1]).values

    input_size = X.shape[1]
    n_samples = X.shape[0]


    W_input_hidden = np.random.randn(input_size, hidden_size) * 0.01
    W_hidden_output = np.random.randn(hidden_size, output_size) * 0.01


    for epoch in range(epochs):

        z1 = np.dot(X, W_input_hidden)
        a1 = relu(z1)

        z2 = np.dot(a1, W_hidden_output)
        a2 = softmax(z2)

        loss = cross_entropy_loss(y, a2)


        dz2 = a2 - y
        dW_hidden_output = np.dot(a1.T, dz2) / n_samples


        da1 = np.dot(dz2, W_hidden_output.T)
        dz1 = da1 * relu_derivative(z1)
        dW_input_hidden = np.dot(X.T, dz1) / n_samples


        W_input_hidden -= lr * dW_input_hidden
        W_hidden_output -= lr * dW_hidden_output


        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss}')


    return W_input_hidden, W_hidden_output

def find_optimal_weights(hidden_layer_size):
    weights_input_hidden, weights_hidden_output = train_neural_network(train_df, hidden_size=hidden_layer_size)
    return [weights_input_hidden, weights_hidden_output]


### Lets start with a simple example

Remember a perceptron takes in data and outputs a single number! Run the next cell to look at our dataset

In [ ]:
labels = ['Spam', 'Spam', 'Not Spam', 'Spam', 'Not Spam', 'Not Spam']
num_words = [75, 76, 68, 85, 66, 71]
spelling_mistake = [12, 16, 3, 17, 5, 8]
num_attachments = [1, 2, 3, 1, 2, 0]
links_embedded = [2, 1, 5, 2, 0, 0]


pre_df = {'labels':labels, 'num_words':num_words, 'num_attachments':num_attachments, 'links_embedded':links_embedded, 'spelling_mistake':spelling_mistake}
df = pd.DataFrame(pre_df)
df

We encounter a new email with the following characteristics! (Run the cell to look at the output in array form)

***NOTE:*** Must run this cell before running the next

In [ ]:
new_email_words = 68 ### number of words in new email
new_email_attachments = 0 ### number of attachments in new email
new_email_links = 3 ### number of links in new email
new_email_spelling_mistakes = 12 ### number of spelling mistakes in new email
new_email = np.array([new_email_words, new_email_attachments, new_email_links, new_email_spelling_mistakes])
new_email #output is in array form

After searching for the best weights for classifying not spam we found them to be the specified ones. Fill in the code to enter the output for the not spam node

In [ ]:
#Run this cell to see the diagram
#If this cell causes an error feel free to move on or ask to see the diagram

Image(filename='notspam.jpg')

In [ ]:
w_not_spam_num_words = .12
w_not_spam_num_attachments = .11
w_not_spam_links_embedded = .08
w_not_spam_spelling = .03
W_not_spam = np.array([w_not_spam_num_words, w_not_spam_num_attachments, w_not_spam_links_embedded, w_not_spam_spelling]) #Array to use if you'd liked
### TODO ####


not_spam_output = ### FILL IN ####

not_spam_output

If you did your calculation correctly you should see 8.76 as your output!

Now lets compute the output of the spam node using the following weights

In [ ]:
#Run this cell to see the diagram
#If this cell causes an error feel free to move on or ask to see the diagram

Image(filename='spam.jpg')

In [ ]:
w_spam_num_words = .15
w_spam_num_attachments = .08
w_spam_links_embedded = .14
w_spam_spelling = .19

W_spam = np.array([w_spam_num_words, w_spam_num_attachments, w_spam_links_embedded, w_spam_spelling]) ### Array to use if youd like
### TODO ####

spam_output = ### FILL IN ###
spam_output

If you did your calculations correctly you should see 12.8999 as your output!

Notice how our two outputs were 8.76 and 12.8999. However we want to represent our outputs as percentages which represent the likelihood that an email is either spam or not spam.

To do this we will put our output through the `softmax` function which takes in a list which contains the outputs and converts them to percentages! We already implemented the softmax() function so call it on the list of outputs to see our predictions

In [ ]:
outputs = [not_spam_output, spam_output]
#output[0] corresponds to not spam and output[1] corresponds to spam
### TODO ###

output_predictions = ### FILL IN ###
output_predictions

Our output shows that our confidence that the email is no spam is 1.5% and the confidence that it is spam is 98%. So we will classify it as ***spam***!

### Now lets try something more involved

For the rest of this assignment we will try to classify photos of numbers using a Neural Network. The following cell will import our data.

In [ ]:
#importing our data
(train_X, train_y), (test_X, test_y) = mnist.load_data()
#printing the shapes of the vectors
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))

### Input images 🖼️

Lets look at what some of our inputs look like

In [ ]:
#Lets look at what our data looks like
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(train_X[i], cmap=plt.get_cmap('gray'))
    plt.show()

As you can see our data is photos of images which are 28 x 28 pixels in size

### Pixel arrays

We can represent these photos by using 2D arrays, each entry in the 2D array will represent a pixel and its value (Black pixels equal 0 and white pixels equal 255)

Lets look at one of these pixel arrays

In [ ]:
train_X[0]

When thinking about how these photos can be applied to our network we can think of each photo being a 784 (28x28) array

Therefore we will have to do this we will flatten our 2D array.

In [ ]:
#Prepping the data
train_X_flattened = train_X.reshape(train_X.shape[0], -1)
column_names = [f'pixel_{i}' for i in range(train_X_flattened.shape[1])] + ['label']
# Combine the image data and labels
train_data = np.column_stack((train_X_flattened, train_y))
#Create dataframe
train_df = pd.DataFrame(train_data, columns=column_names)


test_X_flattened = test_X.reshape(test_X.shape[0], -1)
column_names = [f'pixel_{i}' for i in range(test_X_flattened.shape[1])] + ['label']
# Combine the image data and labels
test_data = np.column_stack((test_X_flattened, test_y))
#Create dataframe
test_df = pd.DataFrame(test_data, columns=column_names)

### Taking a look at our Dataframe

In order to find the optimal weights for our network we will use a lot of prexisting data

In [ ]:
train_df #run this cell to see dataframe

Each row in the above dataframe represent a photo of a number. Since each of the photos are made up of pixels on a 28x28 grid, we can see that the total pixels is 28x28=784 which is one less than the amount of columns in the dataframe. The last column is the label. It is the number corresponding to the drawn image.

### Simplifying calculations using linear algebra ✍️

Instead of doing the calculations for every node indvidually we will multiply each layer at once using matricies!
A matrix is a 2 dimensional array. ***NOTE:*** when we multiply a matrix of n x m dimension with a matrix of m x d dimensions we get a n x d matrix!

In numpy to multiply matrix a by matrix b we perform `np.dot(a,b)`

In [ ]:
# Multiply the following two matricies
# If done correctly output should be
'''
[[27.68594631],
  [25.33169791],
  [22.93721208],
  [26.68534864],
  [25.12106649]]
'''
np.random.seed(63)
a = np.random.rand(5, 100)
b = np.random.rand(100, 1)
### TODO ###

multiplied = ### FILL IN ###
multiplied

### Network Architecture

Now that we have the basics covered lets attempt to make a neural network to predict the numbers! Here's the specifications of the network we are creating:

- Takes in an image of a number which is in the form of an array of size 1 x 784
- Has one hidden layer with 100 nodes (This means our input will be multiplied by an array which will let its output be a 1 x 100 vector)
- Finally from the hidden layer we wil make our predictions which will be represneted by 10 nodes (One for each number) or in other words a 1 x 10 matrix


Instantiate the weights of the correct sizes and then implement the following function which takes in a data point and weights and returns our predictions array

***NOTE:*** np.zeros is a function which takes in two values, `n` and `m`, and returns a matrix which is size n x m

In [ ]:
#image of network architecture
#If this cell causes an error feel free to move on or ask to see the diagram

Image(filename='network_architecture.jpg')

In [ ]:
# In the first set of weights we are taking the input of size 1 x 784 and converting it to a 1 x 100 vector
#fill in the values n and m to make a weight_matrix of size n x m such that the input * weight_matrix = 1 x 100
n = ## FILL IN ###
m = ### FILL IN ###
W_input_to_hidden = np.zeros((n , m))

# In the second set of weights we are taking the hidden layer of size 1 x 100 and converting it to the output
# which is a 1 x 10 matrix
#fill in the values c and d to make a W of size c x d such that the hidden * W = 1 x 10

c = ### FILL IN ###
d = ### FILL IN ###
W_hidden_to_output = np.zeros((c, d))

### Putting it all together
Now that we have the sizes of our weight vectors finalized we have all the pieces to put our network together!

Implement the `run_neural_network` function which takes in the image AND weights and returns the prediction matrix using the neural network

In [ ]:
def run_neural_network(weights, image_vector):
    # input: weights - weights for 1 layer neural network, image_vector - 1D represnetation of 28 x 28 drawing of a number
    # output: 1 x 10 vector representing the predictions of our neural network

    W_input_to_hidden = weights[0]
    W_hidden_to_output = weights[1]
    ### First we will push our data from image_vector to the hidden layer using W_input_to_hidden
    hidden_layer = ### Fill In ###

    #next we will apply an activation function (We will not cover the purpose in this workshop but feel free to search up more about them)
    hidden_layer = relu(hidden_layer)

    ### The next step is to take out data from the hidden layer and push it to the output layer (aka our predictions)
    output_layer = ### Fill In ###

    ### Apply softmax to have our outputs be probabilities
    output_layer = softmax(output_layer)

    return output_layer
weights = [W_input_to_hidden, W_hidden_to_output]
run_neural_network(weights, np.array(train_df.iloc[0][:-1]))

If everything is implemented correctly we should see an output of [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], this is because we have not found the optimal ws for our network, run the following cell to find them (The function uses a technique called gradient descent!)

***NOTE:*** This cell may take 2-5 minutes to run :(

In [ ]:
weights = find_optimal_weights(hidden_layer_size=100)
weights

Now lets try putting an image through the neural network

In [ ]:
#We will try classifying this image
#Lets look at what our data looks like
plt.imshow(train_X[0], cmap=plt.get_cmap('gray'))
plt.show()
np.array(train_df.iloc[0][-1])

In [ ]:
run_neural_network(weights, np.array(train_df.iloc[0][:-1]))

### Testing our accuracy

Now that we have created our neural network lets see how well it performs

Feel free to go back and change the hidden_size in `find_optimal_weights` to improve the accuracy of your model

In [ ]:
def model_accuracy(weights):
    correct = 0
    for i in range(len(test_df)):
        pred = np.argmax(run_neural_network(weights, np.array(train_df.iloc[i][:-1])))
        if pred == train_df.iloc[i][-1]:
            correct += 1
    print(correct/len(test_df))
model_accuracy(weights)

***Run the cell below to draw your own numbers and click the predict number to have the model try to classify your number!***

In [ ]:
# Enable interactive plots
%matplotlib notebook


fig, ax = plt.subplots(figsize=(5, 5))
ax.set_xlim(0, 27)
ax.set_ylim(0, 27)
ax.set_aspect('equal')
plt.xticks([])
plt.yticks([])
plt.grid(True)


out = Output()

x_prev, y_prev = None, None
drawn_rectangles = []


brush_size = 2 # Feel free to change this

# Define a function to handle mouse movement
def on_mouse_move(event):
    global x_prev, y_prev
    if event.button == 1:  # Check if left mouse button is pressed
        x, y = int(event.xdata), int(event.ydata)
        if x_prev is not None and y_prev is not None:
            rect = Rectangle((x - brush_size / 2, y - brush_size / 2), brush_size, brush_size,
                             linewidth=1, edgecolor='black', facecolor='black')
            ax.add_patch(rect)
            drawn_rectangles.append(rect)
            plt.draw()
        x_prev, y_prev = x, y
    else:
        x_prev, y_prev = None, None

fig.canvas.mpl_connect('motion_notify_event', on_mouse_move)

def classify_canv(img):
    return np.argmax(run_neural_network(weights, img.flatten()))
    if np.dot(w_opt,img.flatten()) > 0:
        return num1
    else:
        return num2

def reset_canvas(button):
    global drawn_rectangles, canvas_array
    # Clear drawn rectangles
    for rect in drawn_rectangles:
        rect.remove()
    drawn_rectangles = []
    canvas_array = np.zeros((28, 28))
    plt.draw()

button_reset = Button(description="Reset Canvas")
button_reset.on_click(reset_canvas)



def predict_on_click(button):
    with out:
        out.clear_output()
        # Render the canvas to a numpy array
        canvas_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        canvas_array = canvas_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        # Resize the canvas array to 28x28 shape
        resized_canvas_array = resize(canvas_array, (28, 28), anti_aliasing=True)
        # Convert the resized canvas array to grayscale
        grayscale_canvas_array = rgb2gray(resized_canvas_array)
        inverted_canvas_array = 1 - grayscale_canvas_array
        inverted_canvas_array = (inverted_canvas_array * 255).astype(np.uint8)
        print('Prediction: ' + str(classify_canv(inverted_canvas_array)))

button_predict = Button(description="Predict")
button_predict.on_click(predict_on_click)


display(fig.canvas, button_reset,button_predict, out)
